
# 2.4 — Từ mô hình đến đánh giá(evaluation):

Nhóm xây dựng hai mô hình cơ bản cho biến rain_flag:
1. **Ngưỡng lý thuyết** dựa trên `precipitation_sum` (sanity check): kiểm tra sơ bộ xem dữ liệu có hợp lý không.
2. **Hồi quy logistic** để đưa ra dự báo: dựa trên các thông số đơn giản.


In [9]:
# --- Load config written by 2.2 ---
import json, os, re
from pathlib import Path

def slugify(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "-", str(s).lower()).strip("-")

CONFIG_PATH = "../.lab2_config.json"
CITY = None
city_slug = None

if os.path.exists(CONFIG_PATH):
    with open(CONFIG_PATH, "r", encoding="utf-8") as f:
        cfg = json.load(f)
    CITY = cfg.get("CITY")
    city_slug = cfg.get("city_slug") or (slugify(CITY) if CITY else None)
    print(f"[config] CITY={CITY} | city_slug={city_slug}")
else:
    print("[config] Not found, will auto-detect")


[config] CITY=London | city_slug=london


In [ ]:
# 2.4 Modeling & Evaluation — respect CITY, fallback latest
import os, re
from glob import glob
from pathlib import Path
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import plotly.express as px

PROCESSED_DIR = "../data/processed"
REPORT_DIR = "../reports/figures"

def slugify(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "-", str(s).lower()).strip("-")

# --- OPTIONAL: set CITY here for testing; comment out to auto-pick latest
try:
    CITY  # keep if already set
except NameError:
    CITY = None  # e.g., "Hanoi"

city_slug = slugify(CITY) if CITY else None

# pick features: by city if given, else latest any
if city_slug:
    candidates = sorted(glob(f"../data/processed/{city_slug}/features_*.csv"))
else:
    candidates = sorted(glob("../data/processed/*/features_*.csv"))


feat_path = candidates[-1] if candidates else f"{PROCESSED_DIR}/features.csv"
assert Path(feat_path).exists(), "No features file found. Run 2.3 first."
print("Using features:", feat_path)

df = pd.read_csv(feat_path)

# Derive city_slug from data if needed
if not city_slug:
    if "city" in df.columns and len(df):
        city_slug = slugify(df["city"].iloc[0])
    else:
        city_slug = "unknown"

FIG_DIR = f"{REPORT_DIR}/{city_slug}"
Path(FIG_DIR).mkdir(parents=True, exist_ok=True)

need_cols = {"temp_range","wind_speed_10m_max","dow","rain_flag"}
missing = need_cols - set(df.columns)
assert not missing, f"Missing columns in features file: {missing}"

X = df[["temp_range","wind_speed_10m_max","dow"]].copy()
y = df["rain_flag"].astype(int).copy()
mask = X.notna().all(axis=1) & y.notna()
X, y = X[mask], y[mask]

do_stratify = (y.nunique() == 2) and all((y.value_counts() >= 2))
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y if do_stratify else None
)
print(f"Train size: {len(X_train)} | Test size: {len(X_test)} | Stratify: {do_stratify}")

# Baseline 1: Heuristic
if len(X_test) > 0:
    y_pred_heur = (X_test["wind_speed_10m_max"] > 20).astype(int)
    acc_h = accuracy_score(y_test, y_pred_heur)
    f1_h  = f1_score(y_test, y_pred_heur, zero_division=0)
    cm_h  = confusion_matrix(y_test, y_pred_heur)
    print("\n[Heuristic] ACC:", acc_h, "| F1:", f1_h)
    print("CM:\n", cm_h)
    try:
        fig_cm_h = px.imshow(cm_h, text_auto=True, color_continuous_scale="Blues",
                             labels=dict(x="Pred", y="True", color="Count"),
                             title=f"Confusion Matrix — Heuristic ({city_slug})")
        fig_cm_h.write_image(os.path.join(FIG_DIR, "cm_heuristic.png"))
        fig_cm_h.show()
    except Exception:
        pass

# Baseline 2: Logistic Regression
if len(X_train) >= 1 and y_train.nunique() == 2:
    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1  = f1_score(y_test, y_pred, zero_division=0)
    cm  = confusion_matrix(y_test, y_pred)
    print("\n[Logistic] ACC:", acc, "| F1:", f1)
    print("CM:\n", cm)
    print("\n", classification_report(y_test, y_pred, zero_division=0))

    try:
        proba = clf.predict_proba(X_test)[:, 1]
        px.scatter(x=list(range(len(proba))), y=proba,
                   title=f"P(rain=1) — {city_slug}",
                   labels={"x":"sample idx", "y":"P(rain=1)"}).show()
    except Exception:
        pass

    try:
        px.imshow(cm, text_auto=True, color_continuous_scale="Blues",
                  labels=dict(x="Pred", y="True", color="Count"),
                  title=f"Confusion Matrix — Logistic ({city_slug})").show()
    except Exception:
        pass


Using features: ../data/processed\features_london_20250921_0313.csv
Train size: 4 | Test size: 3 | Stratify: True

[Heuristic] ACC: 0.3333333333333333 | F1: 0.0
CM:
 [[1 1]
 [1 0]]



[Logistic] ACC: 0.3333333333333333 | F1: 0.0
CM:
 [[1 1]
 [1 0]]

               precision    recall  f1-score   support

           0       0.50      0.50      0.50         2
           1       0.00      0.00      0.00         1

    accuracy                           0.33         3
   macro avg       0.25      0.25      0.25         3
weighted avg       0.33      0.33      0.33         3




## Một vài điểm lưu ý:
- Vì 2 điểm **ACC** và **F1** chỉ là baseline, nên khi sự mất cân bằng trong dữ liệu xảy ra, cần sử dụng các model tiên tiến hơn.
- Đây chỉ là bài lab thuần học thuật vì dữ liệu quá ít(7d) và không ổn định.
- Trong dự án sau này, nhóm sẽ sử dụng model hiện đại và sử dụng tập dữ liệu lớn hơn.



## Liên kết tới Lab 3 - Deployment
Sau khi xử lí dữ liệu qua các bước: raw->checkpoint->processed tại đây, nhóm tiếp tục sử dụng chúng cho app minh hoạ trong folder **app/app.py**, đường link dẫn đến Repo: [Lab 3](https://github.com/letrongv4ng/ady201m-group3).
